In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import pymongo
import pdb
from datetime import datetime, timedelta
from dateutil.relativedelta import *
from scipy.io import loadmat
import os
import glob

In [10]:
def transform_lon(lon):
    '''
    Transforms longitude from absolute to -180 to 180 deg
    '''
    if lon >= 180:
        lon -= 360
    return lon

def make_doc(df, date, presLevel, dataVariable, param, measurement, gridName, units):
    '''
    Takes df and converts it into a document for mongodb
    '''
    doc = {}
    df = df.rename(index=str, columns={dataVariable: 'value'})
    dataDict = df.to_dict(orient='records')
    print(gridName)
    doc['gridName'] = gridName
    doc['measurement'] = measurement #temperature or psal
    doc['units'] = units # degrees celsius or psu
    doc['data'] = dataDict 
    doc['variable'] = dataVariable # ARGO_TEMPERATURE_ANOMALY or ARGO_TEMPERATURE_MEAN or total
    doc['date'] = date
    doc['pres'] = float(presLevel)
    doc['param'] = param # anomaly, mean, or total
    doc['cellsize'] = 1  #  Degree
    doc['NODATA_value'] = np.NaN
    return doc

def create_grid_df(chunk):
    df = chunk.to_dataframe()
    df = df.reset_index()
    df = df.rename(columns={'LATITUDE':'lat', 'LONGITUDE':'lon'})
    df['lon'] = df['lon'].apply(lambda lon: transform_lon(lon))
    return df
    
def insert_pres_time_grid(xrDataArray, coll, param, measurement, gridName, units,
                          dataVariable, insertOne=False, meanDf=None):
    for tdx, chunk in xrDataArray.groupby('TIME'):
        month = int(tdx % 12 + 1)
        year = int(2004 + tdx // 12)
        #if not year==2012:
        #    continue
        date = datetime.strptime('{0}-{1}'.format(year, month), '%Y-%m')
        print(date)
        df = create_grid_df(chunk)
        
        if param == 'total':
            df[param] = df['ARGO_TEMPERATURE_ANOMALY'] + meanDf['ARGO_TEMPERATURE_MEAN']
            df=df.drop('ARGO_TEMPERATURE_ANOMALY', axis=1)
        for pdx, presDf in df.groupby('PRESSURE'):
            #if not pdx in [10, 50, 200]:
            #    continue
            presDf = presDf.drop(['TIME', 'PRESSURE'], axis=1)
            doc = make_doc(presDf, date, pdx, dataVariable, param, measurement, gridName, units)
            if insertOne: # Use for testing
                coll.insert_one(doc)
                return
            else:
                coll.insert_one(doc)


In [11]:
def create_collection(dbName, collectionName):
    dbUrl = 'mongodb://localhost:27017/'
    client = pymongo.MongoClient(dbUrl)
    db = client[dbName]
    coll = db[collectionName]
    coll = init_profiles_collection(coll)
    return coll

def init_profiles_collection(coll):
    try:
        coll.create_index([('date', pymongo.ASCENDING)])
        coll.create_index([('pres', pymongo.ASCENDING)])
        coll.create_index([('gridName', pymongo.ASCENDING)])
        coll.create_index([('date', pymongo.ASCENDING), ('pres', pymongo.ASCENDING), ('gridName', pymongo.ASCENDING)])
        coll.create_index([('data.lat', pymongo.DESCENDING)])
        coll.create_index([('data.lon', pymongo.ASCENDING)])
    except:
        logging.warning('not able to get collections or set indexes')
    return coll

In [12]:
meanDf.head()

,PRESSURE,lat,lon,ARGO_TEMPERATURE_MEAN
0,2.5,-64.5,20.5,-1.125
1,2.5,-64.5,21.5,-1.121
2,2.5,-64.5,22.5,-1.121
3,2.5,-64.5,23.5,-1.115
4,2.5,-64.5,24.5,-1.109


In [13]:
rgFilename = '/home/tyler/Desktop/RG_ArgoClim_Temp.nc'
rg = xr.open_dataset(rgFilename, decode_times=False)
meanDf = create_grid_df(rg['ARGO_TEMPERATURE_MEAN'])
dataVariable='ARGO_TEMPERATURE_ANOMALY'
rgDataArray = rg[dataVariable]
param='anomaly'
measurement='temperature'
gridName='rgTempAnom'
units = 'Degrees Celsius'

In [14]:
totalColl = create_collection(dbName='argo', collectionName='rgTempTotal')
totalColl.drop()
insert_pres_time_grid(rgDataArray, totalColl, 'total', measurement, 'rgTempTotal', units,
                          'total', insertOne=False, meanDf=meanDf)

2004-01-01 00:00:00
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
2004-02-01 00:00:00
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal


rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
2005-01-01 00:00:00
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTo

rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
2005-12-01 00:00:00
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
2006-01-01 00:00:00
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal


rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
2006-12-01 00:00:00
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTo

rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
2007-11-01 00:00:00
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
2007-12-01 00:00:00
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal


...

rgTempTotal
rgTempTotal
rgTempTotal
2014-07-01 00:00:00
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
2014-08-01 00:00:00
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal


rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
2015-07-01 00:00:00
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTo

rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
2016-06-01 00:00:00
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
2016-07-01 00:00:00
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal


rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
2017-06-01 00:00:00
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTo

rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
2018-05-01 00:00:00
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
2018-06-01 00:00:00
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal
rgTempTotal


In [15]:
anomColl = create_collection(dbName='argo', collectionName='rgTempAnom')
anomColl.drop()
insert_pres_time_grid(rgDataArray, anomColl, param, measurement, gridName, units,
                          dataVariable, insertOne=False)

2004-01-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
2004-02-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgT

rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
2005-02-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
r

rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
2006-02-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
2006-03-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgT

rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
2007-03-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
r

rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
2008-03-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
2008-04-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgT

...

rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
2014-06-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
2014-07-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgT

rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
2015-07-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
r

rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
2016-07-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
2016-08-01 0

rgTempAnom
rgTempAnom
rgTempAnom
2017-07-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
2017-08-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgT

rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
2018-08-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
rgTempAnom
r

In [19]:
# make for express testing
testColl = create_collection(dbName='argo-express-test', collectionName='rgTempAnom')
#testColl.drop()
insert_pres_time_grid(rgDataArray, testColl, param, measurement, gridName, units,
                          dataVariable, insertOne=True)

2012-01-01 00:00:00
rgTempAnom


# Add mean field

In [5]:
rgFilename = '/home/tyler/Desktop/RG_ArgoClim_Temp.nc'
rg = xr.open_dataset(rgFilename, decode_times=False)
coll = create_collection(dbName='argo', collectionName='rgTempMean')
coll.drop()
dataVariable='ARGO_TEMPERATURE_MEAN'
param='mean'
measurement='temperature'
gridName='rgTemperatureAnomaly'
units = 'Degrees Celsius'

rgDataArray = rg[dataVariable]
insert_pres_time_grid(rgDataArray, coll, param, measurement, gridName, units,
                          dataVariable, insertOne=False)

NameError: name 'insert_pres_time_grid' is not defined